<div align="right" style="text-align:right"><a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Licença Creative Commons" style="border-width:0; float:right" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br><br><i>Prof. Marcelo de Souza</i><br>marcelo.desouza@udesc.br</div>

# Problema da mistura

Programação linear com Pyomo.

---

Uma cervejaria precisa atender um pedido de 100 litros de cerveja com teor alcoólico de 4%. A cervejaria produz uma cerveja do tipo A, com teor alcoólico de 4,5% e custo de \\$0,32 por litro, e uma cerveja do tipo B, com teor alcoólico de 3,7% e custo de $0,25 por litro. Além disso, água é usada na mistura com custo de \\$0,05 por litro. Encontre a mistura de menor custo que atenda os requisitos do cliente.

Modelo:

$
\begin{aligned}
    \text{minimiza} \quad & z = 0,32x_1 + 0,25x_2 + 0,05x_3\\[.3em]
    \text{sujeito a} \quad & x_1 + x_2 + x_3 = 100\\
              & 0,045x_1 + 0,037x_2 = 4\\
              & x_1, x_2, x_3 \ge 0
\end{aligned}
$

---

Preparação para execução no Google Colab

In [ ]:
!pip install -qq pyomo
!apt-get install -y -qq glpk-utils

In [1]:
from pyomo.environ import *

# Criação do modelo
model = ConcreteModel()

# Variáveis de decisão
model.x1 = Var(domain = NonNegativeReals)
model.x2 = Var(domain = NonNegativeReals)
model.x3 = Var(domain = NonNegativeReals)

# Função objetivo
model.obj = Objective(expr = 0.32 * model.x1 + 0.25 * model.x2 + 0.05 * model.x3, sense = minimize)

# Restrições
model.con1 = Constraint(expr = model.x1 + model.x2 + model.x3 == 100)
model.con2 = Constraint(expr = 0.045 * model.x1 + 0.037 * model.x2 == 4)

# Solução
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
opt.solve(model)
print('SOLUÇÃO ÓTIMA')
print('Cerveja A:', model.x1())
print('Cerveja B:', model.x2())
print('Água:', model.x3())
print('Custo:', model.obj())

SOLUÇÃO ÓTIMA
Cerveja A: 37.5
Cerveja B: 62.5
Água: 0.0
Custo: 27.625


---

Na sequência, vamos generalizar o modelo da mistura acima, permitindo a determinação da mistura ótima para qualquer número de componentes (cervejas e outros agentes de mistura, com seus respectivos teores alcoólicos e custos), volume de produção e teor alcoólico desejados. Para isso, definimos algumas variáveis:

+ $V$ é o volume do pedido (em litros de cerveja);
+ $\bar{A}$ é o teor alcoólico desejado;
+ $C$ é o conjunto de componentes (tipos de cerveja, água, ...).

Definimos variáveis para representar o custo por litro e o teor alcoólico de cada componente:

+ $P_c$ é o preço de cada componente $c \in C$;
+ $A_c$ é o teor alcoólico de cada componente $c \in C$.

Finalmente, definimos as variáveis de decisão como a quantidade de litros de cada componente na mistura:

$\qquad\qquad x_c, \forall c \in C$

Com isso, chegamos no seguinte modelo genérico:

$
\begin{aligned}
    \text{minimiza} \quad & z = \sum_{c \in C} x_c P_c\\[.3em]
    \text{sujeito a} \quad & \sum_{c \in C} x_c = V\\
              & \sum_{c \in C} x_c A_c = \bar{A}V\\
              & x_c \ge 0, \forall c \in C
\end{aligned}
$

---

Para implementar o modelo genérico, primeiro definimos os dados do problema, i.e. os valores das variáveis previamente definidas.

In [2]:
# Volume do pedido
V = 100

# Teor alcoólico desejado
A = 0.04

components = {
    'Cerveja A': {'A': 0.045, 'P': 0.32},
    'Cerveja B': {'A': 0.037, 'P': 0.25},
    'Água':      {'A': 0.000, 'P': 0.05},
    #'Vinho':     {'A': 0.083 ,'P': 0.41},
}

Acessamos os dados da seguinte forma:

In [3]:
c = 'Cerveja B' # Componente
print(components[c]['A']) # Teor alcoólico
print(components[c]['P']) # Preço

# O conjunto de componentes pode ser obtido pelas chaves do dicionário de dados
print(components.keys())

0.037
0.25
dict_keys(['Cerveja A', 'Cerveja B', 'Água'])


Finalmente, implementamos e resolvemos o modelo da seguinte forma:

In [4]:
# Lista de componentes
C = components.keys()

# Modelo
model = ConcreteModel()

# Variáveis de decisão: uma para cada componente
model.x = Var(C, domain = NonNegativeReals)

# Função objetivo
model.cost = Objective(expr = sum(model.x[c] * components[c]['P'] for c in C))

# Restrições
model.vol = Constraint(expr = sum(model.x[c] for c in C) == V)
model.alc = Constraint(expr = sum(model.x[c] * components[c]['A'] for c in C) == A * V)

# Solução
solver = SolverFactory('glpk')
solver.solve(model)

print('Mistura ótima')
for c in C:
    print('>', c, ':', model.x[c](), 'litros')
print()
print('Volume =', model.vol(), 'litros')
print('Custo = $', model.cost())

Mistura ótima
> Cerveja A : 37.5 litros
> Cerveja B : 62.5 litros
> Água : 0.0 litros

Volume = 100.0 litros
Custo = $ 27.625
